### Utilizando a base de dados archive.ics.uci.edu/ml/datasets/Car+Evaluation:

In [2]:
import pandas as pd

data = pd.read_csv('car.csv')
data.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


### 1. (10 pontos) Defina o classificador cego com o menor erro de classificação possível. 

### 2. (18 pontos) Calcule as probablidades a priori para cada variável (para cada possível valor de cada variável).

### 3. (18 pontos) Calcule a probabilidade de cada valor de cada variável dado a classe (posteriori).

### 4. (18 pontos) Implemente o classificador Naive Bayes.

### 5. (18 pontos) Utilizando sua implementação e metade dos exemplos de cada classe como conjunto de treino e o resante como conjunto de teste, avalie o classificador Naive Bayes.

### 6. (18 pontos) Faça o mesmo da questão anterior utilizando uma implementação de biblioteca com configuração semelhente ao classificador solicitado na questão anterior.